# Trade Analytics Notebook
## Import dependencies & services

In [1]:
import pandas as pd
import plotly.graph_objects as go

from tradebot.infrastructure.mt5_history import MT5HistoryRepository
from tradebot.application.position_service import PositionService


## Load raw positions into a DataFrame

In [2]:
def load_positions() -> pd.DataFrame:
    """Fetch MT5 positions and return a tidy DataFrame."""
    repo = MT5HistoryRepository()
    svc  = PositionService(repo)
    return pd.DataFrame(svc.positions())

positions = load_positions()
positions.to_csv("positions.csv", index=False)   # optional cache / debug


2025-06-04 23:44:22.049 | INFO     | tradebot.infrastructure._mt5_utils:ensure_mt5:17 - MT5  already logged in (account 7303680). No login attempt.
2025-06-04 23:44:22.050 | INFO     | tradebot.infrastructure.mt5_history:__init__:12 - MT5 history repository ready
c:\Zeno\TeleTrader\tradebot\application\position_service.py:36: RuntimeWarning: invalid value encountered in scalar divide
  vwap = lambda d: (d["price"]*d["volume"]).sum()/d["volume"].sum()


## Aggregate trades

In [3]:
def count_positive(series: pd.Series) -> int:
    """Helper: # of strictly positive values in a Series."""
    return (series > 0).sum()

def group_trades(df: pd.DataFrame) -> pd.DataFrame:
    """Group by (trader × time_open) and compute summary statistics."""
    grouped = (
        df.groupby(['trader', 'time_open'])
          .agg(
              sum_profit        = ('profit',        'sum'),
              mean_duration     = ('duration_sec',  'mean'),
              total_target      = ('target_total',  'first'),
              successful_targets= ('profit',        count_positive),
              min_rr            = ('rr',            'min'),
              max_rr            = ('rr',            'max'),
          )
          .reset_index()
          .rename(columns={'time_open': 'time'})
    )

    grouped['time'] = pd.to_datetime(grouped['time'], utc=True)
    return (grouped
            .sort_values('time')
            .set_index('time')        # ← makes resampling very easy
           )

grouped = group_trades(positions)
grouped.head()        # sanity-check


,trader,sum_profit,mean_duration,total_target,successful_targets,min_rr,max_rr
time,,,,,,,
2025-05-26 18:46:54+00:00,Lily,-96.60,22587.000000,3.0,0,0.62,1.62
2025-05-27 04:40:01+00:00,Eli,0.00,NaN,1.0,0,1.50,1.50
2025-05-27 05:41:12+00:00,Nemat,83.42,8692.666667,6.0,6,0.25,1.38
2025-05-27 08:45:49+00:00,Lily,-95.43,43.000000,3.0,0,0.93,2.40
2025-05-27 10:21:43+00:00,Nemat,55.06,3769.400000,5.0,5,0.13,1.19


##  Weekly trade-count bar chart

In [4]:
def weekly_trade_count_plot(gdf: pd.DataFrame) -> go.Figure:
    """Interactive bar chart: # trades per week, filterable by trader."""
    weekly = (
        gdf.groupby('trader')
           .resample('W-MON')['sum_profit']
           .count()
           .rename('trade_count')
           .reset_index()
    )
    pivot = weekly.pivot(index='time', columns='trader', values='trade_count').fillna(0)

    fig = go.Figure([go.Bar(x=pivot.index, y=pivot[c], name=c) for c in pivot])

    # --- dropdown buttons ----------------------------------------------------
    btns = [
        dict(label="All Traders",
             method="update",
             args=[{"visible": [True]*len(pivot.columns)},
                   {"title": "Weekly Trade Count – All Traders"}])
    ]
    for i, trader in enumerate(pivot.columns):
        vis = [False]*len(pivot.columns); vis[i] = True
        btns.append(
            dict(label=trader,
                 method="update",
                 args=[{"visible": vis},
                       {"title": f"Weekly Trade Count – {trader}"}])
        )

    fig.update_layout(
        title="Weekly Trade Count – All Traders",
        xaxis_title="Week (Monday close)",
        yaxis_title="Number of Trades",
        updatemenus=[dict(buttons=btns, x=1.15, y=1.1, showactive=True)],
        height=500
    )
    return fig

weekly_trade_count_plot(grouped).show()


## Cumulative P&L plot 

In [5]:
def cumulative_pnl_plot(gdf: pd.DataFrame) -> go.Figure:
    """Interactive line plot: cumulative P&L vs time OR vs trade #."""
    traders   = sorted(gdf['trader'].dropna().unique().tolist())
    categories= ['All Traders'] + traders

    traces, idx = [], {}      # map (trader, mode) → trace index

    for trader in categories:
        data = gdf if trader == 'All Traders' else gdf[gdf['trader'] == trader]
        if data.empty: 
            continue
        data = data.copy()
        data['cum_profit']   = data['sum_profit'].cumsum()
        data['trade_number'] = range(1, len(data)+1)

        # --- time-based trace
        idx[(trader, 'time')] = len(traces)
        traces.append(go.Scatter(
            x=data.index, y=data['cum_profit'],
            name=f"{trader} (Time)",
            mode='lines+markers',
            visible=(trader == 'All Traders')
        ))
        # --- count-based trace
        idx[(trader, 'count')] = len(traces)
        traces.append(go.Scatter(
            x=data['trade_number'], y=data['cum_profit'],
            name=f"{trader} (Count)",
            mode='lines+markers',
            visible=False
        ))

    # --- dropdown A: trader filter
    trader_buttons = []
    for trader in categories:
        vis = [False]*len(traces)
        if (trader, 'time') in idx:
            vis[idx[(trader, 'time')]] = True
        trader_buttons.append(dict(
            label=trader, method="update",
            args=[{"visible": vis},
                  {"title": f"Cumulative Profit – {trader} (Time)", 
                   "xaxis": {"title": "Time"}}]
        ))

    # --- dropdown B: x-axis mode
    mode_buttons = [
        dict(label="X: Time", method="update",
             args=[{"visible": [t.name.endswith('(Time)') for t in traces]},
                   {"xaxis": {"title": "Time"}}]),
        dict(label="X: Trade #", method="update",
             args=[{"visible": [t.name.endswith('(Count)') for t in traces]},
                   {"xaxis": {"title": "Trade Number"}}])
    ]

    fig = go.Figure(traces)
    fig.update_layout(
        title="Cumulative Profit – All Traders (Time)",
        yaxis_title="Cumulative Profit",
        xaxis_title="Time",
        updatemenus=[
            dict(buttons=trader_buttons, direction="down", x=1.16, y=0.85),
            dict(buttons=mode_buttons,   direction="down", x=1.16, y=0.72),
        ],
        height=600,
        legend=dict(orientation="h", yanchor="bottom", y=-0.25)
    )
    return fig

cumulative_pnl_plot(grouped).show()
